In [ ]:
import sys, os
import numpy as np
from scipy.stats import zscore
from neuropop import dimensionality

# path to paper code
sys.path.insert(0, '/github/rastermap/paper')
from loaders import load_fish_data, load_visual_data, load_alexnet_data

# path to directory with data etc
### *** CHANGE THIS TO WHEREEVER YOU ARE DOWNLOADING THE DATA ***
root = "/media/carsen/ssd2/rastermap_paper/"
# (in this folder we have a "data" folder and a "results" folder)


### compute SVCA for large datasets


In [ ]:
scovs = []

dat = np.load(os.path.join(root, "data/", "spont_data.npz"))
spks = dat["spks"]
spks = zscore(spks, axis=1)
scov, varcov = dimensionality.SVCA(spks)
scovs.append(scov)

dat = np.load(os.path.join(root, "data/", "corridor_neur.npz"))
xpos, ypos, spks = dat["xpos"], dat["ypos"], dat["spks"]
spks = zscore(spks, axis=1)
scov, varcov = dimensionality.SVCA(spks)
scovs.append(scov)

### folder with "subject_8" folder
data_path = os.path.join(root, "data")
spks, F, xyz, stims, swimming, eyepos = load_fish_data(data_path, subject=8)
spks = zscore(spks, axis=1)
scov, varcov = dimensionality.SVCA(spks)
scovs.append(scov)

filename = os.path.join(root, "data/", "TX61_3x.npz")
stim_filename = os.path.join(root, "data/", "text5k_3x.mat")
out = load_visual_data(filename, stim_filename)
spks = out[0]
spks = zscore(spks, axis=1)
scov, varcov = dimensionality.SVCA(spks)
scovs.append(scov)

env_ids = ["PongNoFrameskip-v4", "SpaceInvadersNoFrameskip-v4", 
           "EnduroNoFrameskip-v4", "SeaquestNoFrameskip-v4"]

for env_id in env_ids:
    dat = np.load(os.path.join(root, "simulations/", f"qrdqn_{env_id}.npz"))
    spks = dat["spks"]

    x_std = spks.std(axis=1)
    igood = x_std > 1e-3
    print(igood.mean())

    S = zscore(spks[igood], axis=1)
    scov, varcov = dimensionality.SVCA(S)
    scovs.append(scov)

filename = os.path.join(root, "data", "ann_fvs_Grayscale(224)_TX61_3X.npz")
sresp, ilayer, ipos, iconv, nmax = load_alexnet_data(filename)
for l in range(5):
    S = zscore(sresp[ilayer==l].copy(), axis=1)
    scov, varcov = dimensionality.SVCA(S)
    scovs.append(scov)

scovs_all = np.nan * np.zeros((13, 1024))
for k, scov in enumerate(scovs):
    scovs_all[k, :len(scov)] = scov

np.save(os.path.join(root, "results", "scovs.npy"), scovs_all)

### make figure

In [ ]:
from fig_utils import *

scovs_all = np.load(os.path.join(root, "results", "scovs.npy"))

fig = plt.figure(figsize=(14, 7.5))
grid = plt.GridSpec(3,5, figure=fig, left=0.06, right=0.98, top=0.96, bottom=0.08, 
                    wspace = 0.8, hspace = 0.8)
transl = mtransforms.ScaledTranslation(-13 / 72, 20 / 72, fig.dpi_scale_trans)
il = 0

titles = ["spontaneous activity, mouse", "virtual reality, mouse", "zebrafish wholebrain activity", "flashed images, mouse",
    "QRDQN - Pong", "QRDQN - SpaceInvaders", "QRDQN - Enduro", "QRDQN - Seaquest",
    "Alexnet - layer 1", "Alexnet - layer 2", "Alexnet - layer 3", "Alexnet - layer 4", "Alexnet - layer 5"]

for k, scov in enumerate(scovs_all):
    if k<8:
        if k==2 or k==3:
            ki = 3 if k==2 else 2
        else:
            ki = k
        ax = plt.subplot(grid[ki//4, ki%4])
    else:
        ax = plt.subplot(grid[2, k-8])
    ss = scov.copy()
    ss /= ss[0]
    #ss *= len(ss)
    alpha, ypred = dimensionality.get_powerlaw(ss, np.arange(11, 500))
    #print(alpha)
    ax.loglog(np.arange(1, len(ss)+1), ss, color=[0,0.5,1])
    ax.plot(np.arange(len(ypred))+1, ypred, color="k", lw=1)
    ax.text(0.5, 0.7, rf"$\alpha$={alpha:.2f}", transform=ax.transAxes)
    ax.set_title(titles[k], fontsize="medium")#, fontsize="small")
    ax.set_ylim([1/1024, 1.5])
    ax.set_xlim([1, 1024])
    ax.set_xticks([1, 10, 100, 1000])
    ax.set_xlabel("PC #")
    ax.set_ylabel("shared covariance")
    ax.xaxis.get_minor_locator().set_params(numticks=99, subs=np.arange(0.1, 1, 0.1))

fig.savefig(os.path.join(root, "figures", "suppfig_powerlaws.pdf"))